# Gradient Descent with Momentum & RMSprop (NumPy)

This notebook demonstrates **gradient descent**, **gradient descent with momentum**, and **RMSprop** using pure NumPy on a simple 2D optimization problem.

內容大綱：

1. 定義一個簡單但「不等方」的二次函數（讓震盪現象明顯）  
2. 實作：
   - 標準梯度下降 (Vanilla Gradient Descent)
   - 帶動量的梯度下降 (Momentum)
   - RMSprop
3. 比較不同演算法的收斂軌跡與效果


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# For nicer printing
np.set_printoptions(precision=4, suppress=True)

## 1. Toy Objective Function

我們使用一個簡單的 2D 二次函數：

\[
f(w_1, w_2) = a w_1^2 + b w_2^2
\]

其中：

- 若 \(a \ll b\)，則在 \(w_1\) 方向較平，在 \(w_2\) 方向較陡  
- 這會導致普通梯度下降在陡峭方向劇烈震盪，在平坦方向移動很慢 → 非常適合展示 Momentum 和 RMSprop 的效果。


In [ ]:
# Quadratic function parameters (ill-conditioned)
a = 0.1  # flatter direction
b = 5.0  # steeper direction

def f(w):
    """Objective function: f(w1, w2) = a*w1^2 + b*w2^2"""
    w1, w2 = w
    return a * w1**2 + b * w2**2

def grad_f(w):
    """Gradient of f with respect to w = [w1, w2]."""
    w1, w2 = w
    df_dw1 = 2 * a * w1
    df_dw2 = 2 * b * w2
    return np.array([df_dw1, df_dw2])

## 2. 可視化工具：等高線與軌跡

我們定義一個輔助函式，用來在 2D 平面畫出：

- 目標函數的等高線 (contours)  
- 優化過程中每一步的參數位置（軌跡）


In [ ]:
def plot_contours_with_trajectory(trajectory, title="Trajectory", levels=30):
    trajectory = np.array(trajectory)
    w1_vals = trajectory[:, 0]
    w2_vals = trajectory[:, 1]

    # 建立網格
    w1_min, w1_max = w1_vals.min() - 1.0, w1_vals.max() + 1.0
    w2_min, w2_max = w2_vals.min() - 1.0, w2_vals.max() + 1.0

    W1, W2 = np.meshgrid(
        np.linspace(w1_min, w1_max, 200),
        np.linspace(w2_min, w2_max, 200)
    )
    Z = a * W1**2 + b * W2**2

    plt.figure(figsize=(6, 5))
    plt.contour(W1, W2, Z, levels=levels)
    plt.plot(w1_vals, w2_vals, marker="o")
    plt.scatter([0], [0])
    plt.title(title)
    plt.xlabel("w1")
    plt.ylabel("w2")
    plt.grid(True)
    plt.show()

## 3. 標準梯度下降 (Vanilla Gradient Descent)

更新規則：

\[
w := w - \alpha \, \nabla f(w)
\]

其中：

- \(\alpha\) 是學習率 (learning rate)
- \(\nabla f(w)\) 是梯度


In [ ]:
def gradient_descent(w_init, alpha=0.05, num_iters=50):
    w = w_init.copy().astype(float)
    trajectory = [w.copy()]
    costs = [f(w)]
    for i in range(num_iters):
        grad = grad_f(w)
        w = w - alpha * grad
        trajectory.append(w.copy())
        costs.append(f(w))
    return np.array(trajectory), np.array(costs)

# Run vanilla GD from a starting point
w0 = np.array([3.0, 3.0])  # initial point
traj_gd, costs_gd = gradient_descent(w0, alpha=0.05, num_iters=50)
print("Final w (GD):", traj_gd[-1])
print("Final cost (GD):", costs_gd[-1])

plot_contours_with_trajectory(traj_gd, title="Vanilla Gradient Descent")

## 4. 帶動量的梯度下降 (Momentum)

Momentum 的更新規則如下（使用指數加權平均）：

1. 動量更新：

\[
v := \beta v + (1 - \beta) \, \nabla f(w)
\]

2. 參數更新：

\[
w := w - \alpha v
\]

其中：

- \(v\)：梯度的指數加權平均（類似「速度」）  
- \(\beta\)：動量超參數（常用 0.9）  
- \(\alpha\)：學習率  


In [ ]:
def gradient_descent_momentum(w_init, alpha=0.05, beta=0.9, num_iters=50):
    w = w_init.copy().astype(float)
    v = np.zeros_like(w)
    trajectory = [w.copy()]
    costs = [f(w)]
    for i in range(num_iters):
        grad = grad_f(w)
        v = beta * v + (1 - beta) * grad
        w = w - alpha * v
        trajectory.append(w.copy())
        costs.append(f(w))
    return np.array(trajectory), np.array(costs)

traj_mom, costs_mom = gradient_descent_momentum(w0, alpha=0.1, beta=0.9, num_iters=50)
print("Final w (Momentum):", traj_mom[-1])
print("Final cost (Momentum):", costs_mom[-1])

plot_contours_with_trajectory(traj_mom, title="Gradient Descent with Momentum")

## 5. RMSprop

RMSprop 會對「梯度平方」做指數加權平均，並用來調整每一個維度的「有效步長」。

更新規則：

1. 累積平方梯度的指數加權平均：

\[
S := \beta_2 S + (1 - \beta_2) (\nabla f(w))^2
\]

> 上式中的平方與加法都是 **逐元素 (element-wise)**。

2. 參數更新：

\[
w := w - \alpha \frac{\nabla f(w)}{\sqrt{S} + \varepsilon}
\]

- \(\beta_2\)：RMSprop 的衰減係數（常用 0.9、0.99 等）  
- \(\varepsilon\)：一個很小的常數，例如 \(10^{-8}\)，避免除以 0


In [ ]:
def rmsprop(w_init, alpha=0.05, beta2=0.9, epsilon=1e-8, num_iters=50):
    w = w_init.copy().astype(float)
    S = np.zeros_like(w)
    trajectory = [w.copy()]
    costs = [f(w)]
    for i in range(num_iters):
        grad = grad_f(w)
        # Exponentially weighted average of squared gradients
        S = beta2 * S + (1 - beta2) * (grad ** 2)
        # Parameter update (element-wise division)
        w = w - alpha * grad / (np.sqrt(S) + epsilon)
        trajectory.append(w.copy())
        costs.append(f(w))
    return np.array(trajectory), np.array(costs)

traj_rms, costs_rms = rmsprop(w0, alpha=0.1, beta2=0.9, epsilon=1e-8, num_iters=50)
print("Final w (RMSprop):", traj_rms[-1])
print("Final cost (RMSprop):", costs_rms[-1])

plot_contours_with_trajectory(traj_rms, title="RMSprop")

## 6. 收斂速度比較

我們將三種方法在同樣起點、類似的學習率設定下的 **loss 曲線** 畫在一起，觀察：

- 哪個下降得比較快  
- 哪個震盪比較少  


In [ ]:
# 使用相同起點重新跑，便於比較
w0 = np.array([3.0, 3.0])

traj_gd, costs_gd = gradient_descent(w0, alpha=0.05, num_iters=80)
traj_mom, costs_mom = gradient_descent_momentum(w0, alpha=0.1, beta=0.9, num_iters=80)
traj_rms, costs_rms = rmsprop(w0, alpha=0.1, beta2=0.9, epsilon=1e-8, num_iters=80)

plt.figure(figsize=(7, 5))
plt.plot(costs_gd, label="GD")
plt.plot(costs_mom, label="Momentum")
plt.plot(costs_rms, label="RMSprop")
plt.yscale("log")  # 常用 log scale 觀察收斂速度
plt.xlabel("Iteration")
plt.ylabel("Cost (log scale)")
plt.title("Convergence Comparison")
plt.legend()
plt.grid(True)
plt.show()

## 7. 小結

- **Vanilla Gradient Descent**：
  - 同一學習率作用在所有維度上  
  - 在「陡峭方向」容易震盪，為了避免發散，必須用很小的學習率 → 收斂變慢  

- **Momentum**：
  - 對梯度做指數加權平均，像是讓參數有「速度」  
  - 沿著一致方向會越滾越快，在來回震盪方向會互相抵消  
  - 通常能明顯加快收斂，減少 Zig-Zag 路徑  

- **RMSprop**：
  - 對「梯度平方」做指數加權平均  
  - 在梯度一直很大的方向，自動縮小步伐；在梯度小的方向，相對放大步伐  
  - 等同於為每個參數維度設定「自適應學習率」  

後續若再結合 Momentum 與 RMSprop，就可以得到常用的 **Adam** 演算法。